In [29]:
real_window_size_pos = 9
real_window_size_np_chunk = 0
real_window_size_n_dep = 5
real_window_size_depth = 5
real_window_size_n_siblings = 5 

using_pos = 0
#using_chunking = 1
using_parsing = 1
#using_dependency = 1
#################################################################################################################
window_size_pos = (real_window_size_pos * 2) + 1
window_size_np_chunk = (real_window_size_np_chunk * 2) + 1
window_size_n_dep = (real_window_size_n_dep * 2) + 1
window_size_depth = (real_window_size_depth * 2) + 1
window_size_n_siblings = (real_window_size_n_siblings * 2) + 1


import collections
import nltk
from nltk import pos_tag
from nltk.corpus import treebank
from data_handler import *
from nltk import chunk
from feature_design_for_linguistic import *

from chunkers import ClassifierChunker
from nltk.corpus import treebank_chunk 
from chunkers import TagChunker 
train_chunks = treebank_chunk.chunked_sents()[:3000] # training data
# test_chunks = treebank_chunk.chunked_sents()[3000:] # testing data
# score = chunker.evaluate(test_chunks)
# score.accuracy() # 0.9721733155838022
chunker = ClassifierChunker(train_chunks) # a classifier trained by treebank dataset

import os
os.environ['STANFORD_PARSER'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser.jar'
os.environ['STANFORD_MODELS'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser-3.7.0-models.jar'
from nltk.parse.stanford import StanfordParser
parser = StanfordParser(model_path='edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz')

from nltk.parse.stanford import StanfordDependencyParser
dep_parser = StanfordDependencyParser(model_path="edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz")

#################################################################################################################
# pre_X_train = load('pre_X_train') # preprocessed text data
# pre_X_test = load('pre_X_test') # preprocessed text data
pre_X_train = load('X_train') 
pre_X_test = load('X_test') 
Y_train = load('pre_Y_train')

### FE2_X_train와 FE2_X_test 초기화
FE2_X_train = []
for sentence in (pre_X_train):
    temp1 = []
    for token in (sentence):
        temp1.append([])
    FE2_X_train.append(temp1)

FE2_X_test = []
for sentence in (pre_X_test):
    temp2 = []
    for token in (sentence):
        temp2.append([])
    FE2_X_test.append(temp2)

C:\Users\Office\Anaconda3\lib\site-packages\nltk\tokenize\regexp.py:123: FutureWarning: split() requires a non-empty pattern match.
  return [tok for tok in self._regexp.split(text) if tok]


## 1. Part of speech (POS)

* POS feature 역시 사전-based feature와 같이 binary feature로 구성하면 될 것 같다. NLTK에서 제공하는 POS 종류는 45개가 있다. 그런데, 45종류를 모두 포함해버리면 또 sparsity problem이 생길 것 같다. (물론 여태까지 그런 문제가 발생하기 보다는 성능이 좋아졌지만...)
* 일단은 모두 포함하는 걸로 하고, 나중에 best feature set을 찾을 때, 비슷한 것끼리 묶는 법도 생각해보자.
* 다른 사전들과는 다르게 POS 사전은 unknown_token이 없다.
* 45개 모두 사용한 POS 사전의 성능은 41%정도 밖에 안된다. information gain이 크진 않다. 나중에 45개 범주를 더 줄이는 방안도 고려해봐야겠다.
* window size를 적용해보자.

In [30]:
###############################
""" CREATE POS LOOKUP TABLE """
###############################
# voca_set_POS = ['$', "''", '(', ')', ',', '--', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 
#                 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM',
#                 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
list_POS = [
    'DT','JJ','NN','MD','VB',',','CC','IN','.','VBG','VBZ','VBN','TO','WRB','NNS','RB','PRP','VBP','PRP$','CD',
    'NNP','EX','VBD','JJR',':','WDT','RP','NNPS','POS',"''",'RBR','WP','PDT','JJS','RBS','$','FW'
]

lookup_table_POS = { }
for i, token in enumerate(list_POS):
    lookup_table_POS[token] = i # POS에서는 1부터(i+1) 시작안해도 된다. POS에는 unknown이 없기 때문이다.
# Add unknown token
print('len(lookup_table_POS) = ', len(lookup_table_POS))

##############################
""" POS FEATURE EXTRACTION """
##############################
if using_pos == 1:    
    # For training data
    for i, sentence in enumerate(pre_X_train):
        pos_tagged_sentence = nltk.pos_tag(sentence)
        #print(pos_tagged_sentence)
        for j, tuple_ in enumerate(pos_tagged_sentence): # pos_sentence
            FE2_X_train[i][j] += FE_POS(j, pos_tagged_sentence, window_size_pos, lookup_table_POS)        
            
    # For testing data
    for i, sentence in enumerate(pre_X_test):
        pos_tagged_sentence = nltk.pos_tag(sentence)
        for j, tuple_ in enumerate(pos_tagged_sentence): # pos_sentence
            FE2_X_test[i][j] += FE_POS(j, pos_tagged_sentence, window_size_pos, lookup_table_POS)

    print('window size pos = ', window_size_pos)
    print('===> current feature length', len(FE2_X_train[0][0]))
    len_pos_feature = len(FE2_X_train[0][0])
    assert(len(FE2_X_train[0][0]) == len(FE2_X_test[0][0]))

len(lookup_table_POS) =  37


## 2. Chunking (shallow parsing)

* NP를 기반으로 window size = 3으로 binary value로 feature를 구성해보자.

Parse tree 구조는 다음과 같다. <br>
가장 작은 단위로 묶는 단위는 POS tag이다. e.g., IN, VBP, DT, NN 등등 <br>
그 다음으로 큰 단위는 chunk 단위이다. e.g, NP (Noun Phrase)
chuck 단위에서 왜 VP (Verb Phrase)와 PP (Prepositional Phrase)는 언급하지 않나? 이유는 VP와 PP는 NP를 포함할 수 있기 때문이다.
따라서, NP와 동등한 단위가 될 수 없다. 물론 NP가 VP 또는 PP와 동등한 단위가 될 수도 있다. 여기서 이야기 하는 바는 최소한의 단위이다. <br>
NP가 가장 작은 단위의 chunk이고, PP와 VP는 그 상위 레벨에 있는 기준이다. 따라서 이들과 구분하는 것이 좋다. <br>
따라서 나는 PP와 VP는 chunking의 다음 단계인 full parsing에서 다루도록 하겠다. <br><br>

NLTK 라이브러리에 있는 chunking 도구 중에서 규칙으로 (사람이 메뉴얼로 작성) 만들 수 있는 chunking이 있다. 여기에 VP, NP, PP를 구성할 수도 있다. 이렇게되면 chunking으로 VP, NP, PP를 모두 추출할 수 있는데, 문제는 성능이 낮다. (60%) 반면, 일반적인 (기계학습기반 그리고 NP만 추출하는..) chunking 도구들의 성능은 95%~97%정도 한다.




In [31]:
# if using_chunking == 1:
#     #########################
#     """ For training data """
#     #########################
#     for i, sentence in enumerate(pre_X_train):
#         pos_tagged_sentence = nltk.pos_tag(sentence)
#         tree = chunker.parse(pos_tagged_sentence)
#         print(tree)
        
#         NP_word_list = [word[0] for word,pos in tree.pos() if pos=='NP']
#         print(NP_word_list)
#         #for j, token in enumerate(sentence):
#         #    FE2_X_train[i][j] += FE_NP_CHUNK(j, sentence, NP_word_list, window_size_np_chunk)
#         print('============================================================================================')
#         print('\n\n')
            
#     ########################
#     """ For testing data """
#     ########################
#     for i, sentence in enumerate(pre_X_test):
#         pos_tagged_sentence = nltk.pos_tag(sentence)
#         tree = chunker.parse(pos_tagged_sentence)
#         NP_word_list = [word[0] for word,pos in tree.pos() if pos=='NP']
#         for j, token in enumerate(sentence):
#             FE2_X_test[i][j] += FE_NP_CHUNK(j, sentence, NP_word_list, window_size_np_chunk)

#     print('window size np chunk = ', window_size_np_chunk)
#     print('===> current feature length', len(FE2_X_train[0][0])-len_pos_feature)
#     len_chunk_feature = len(FE2_X_train[0][0])-len_pos_feature
#     assert(len(FE2_X_train[0][0]) == len(FE2_X_test[0][0]))

## 3. (Full) Parsing, (Constituency based Parsing Tree)

NLTK는 full parsing과 dependency parsing이 규칙기반으로 이뤄진 모델밖에 없어서. 여기서부터는 다른 라이브러리를 사용한다. <br>
기본적으로 NLTK는 noun phrase chunking까지만 제공해준다. 그 이후의 단계 (full parsing)는 규칙을 이용해서 만든다. <br>

Parsing은 stanford NLP에서 제공해주는 parser를 사용할 것이고, java로 구현되어 있는 stanford parser를 NLTK가 python으로 인터페이스해준다.

PP, NP, VP를 추가하면 성능이 조금 떨어진 이유를 생각해보면.
depth, sibling과 correlation이 있기 때문이다
이건 사실이다. 둘 다 비슷한 feature이다.

In [32]:
### Features for several phrases
def initial_tree_parameter(temp_sentence):

    global list_current_depth
    global previous_height
    global current_height
    global previous_depth
    global current_depth
    global phrase_list_until_leaf
    global per_token
    global depth_cnt
    
    ######
    global index_NPSBAR
    global index_VPSBAR
    global index_SSBAR
    global index_SQSBAR
    
    global index_PPPP
    global index_VPPP
    global index_NPPP
    
    global index_NPNP
    global index_VPNP
    global index_SNP
    
    global index_VPVP
    global index_SVP
    
    ######
    global list_NPSBAR
    global list_VPSBAR
    global list_SSBAR
    global list_SQSBAR
    
    global list_PPPP
    global list_VPPP
    global list_NPPP
    
    global list_NPNP
    global list_VPNP
    global list_SNP
    
    global list_VPVP
    global list_SVP
    
    list_NPSBAR = [0] * len(temp_sentence)
    list_VPSBAR = [0] * len(temp_sentence)
    list_SSBAR = [0] * len(temp_sentence)
    list_SQSBAR = [0] * len(temp_sentence)
    
    list_PPPP = [0] * len(temp_sentence)
    list_VPPP = [0] * len(temp_sentence)
    list_NPPP = [0] * len(temp_sentence)
    
    list_NPNP = [0] * len(temp_sentence)
    list_VPNP = [0] * len(temp_sentence)
    list_SNP = [0] * len(temp_sentence)
    
    list_VPVP = [0] * len(temp_sentence)
    list_SVP = [0] * len(temp_sentence)
    
    phrase_list_until_leaf = []
    per_depth = []
    per_token = []
    list_current_depth = []
    previous_height = 0
    current_height = 0
    previous_depth = 0
    current_depth = 0
    depth_cnt = 0
    
    index_NPSBAR = -1
    index_VPSBAR = -1
    index_SSBAR = -1
    index_SQSBAR = -1
    
    index_PPPP = -1
    index_VPPP = -1
    index_NPPP = -1
    
    index_NPNP = -1
    index_VPNP = -1
    index_SNP = -1
    
    index_VPVP = -1
    index_SVP = -1


def traverseTree(tree):

    global list_current_depth
    global previous_height
    global current_height
    global previous_depth
    global current_depth
    global phrase_list_until_leaf
    global per_token
    global depth_cnt
    global temp_sentence
    
    #####
    global index_NPSBAR
    global index_VPSBAR
    global index_SSBAR
    global index_SQSBAR
    
    global index_PPPP
    global index_VPPP
    global index_NPPP
    
    global index_NPNP
    global index_VPNP
    global index_SNP
    
    global index_VPVP
    global index_SVP
       
    ######
    global list_NPSBAR
    global list_VPSBAR
    global list_SSBAR
    global list_SQSBAR
    
    global list_PPPP
    global list_VPPP
    global list_NPPP
    
    global list_NPNP
    global list_VPNP
    global list_SNP
    
    global list_VPVP
    global list_SVP
    
    
    if type(tree) == nltk.tree.Tree:
        
        # Initialication
        current_height = tree.height()
        
        
        # For SBAR, NPPP, VPPP, NP
#         if tree.label() == 'SBAR':
#             tokens_SBAR = tree.leaves()
# #             print(len(tokens_SBAR))
# #             print(len(temp_sentence))
# #             print(tokens_SBAR)
# #             print(temp_sentence)
# #             print("**************************")
#             for i, token in enumerate(temp_sentence):
#                 if temp_sentence[i] == tokens_SBAR[0]:
#                     if temp_sentence[i+len(tokens_SBAR)-1] == tokens_SBAR[-1]:
#                         index_SBAR = i
#                         break
            
#             if not index_SBAR == -1:
#                 for j in range(index_SBAR, index_SBAR + len(tokens_SBAR)):
#                     list_SBAR[j] = 1

        # NP - SBAR
        if tree.label() == 'SBAR' and list_current_depth[-1] == 'NP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_NPSBAR = i
                    break    
            if not index_NPSBAR == -1:
                for j in range(index_NPSBAR, index_NPSBAR+len(tree.leaves())):
                    list_NPSBAR[j] = 1              
        
        # VP - SBAR
        elif tree.label() == 'SBAR' and list_current_depth[-1] == 'VP':
            
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_VPSBAR = i
                    break   
            if not index_VPSBAR == -1:
                for j in range(index_VPSBAR, index_VPSBAR+len(tree.leaves())):
                    list_VPSBAR[j] = 1         
        
        # S - SBAR
        elif tree.label() == 'SBAR' and list_current_depth[-1] == 'S':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_SSBAR = i
                    break    
            if not index_SSBAR == -1:
                for j in range(index_SSBAR, index_SSBAR+len(tree.leaves())):
                    list_SSBAR[j] = 1      
        
        # SQ - SBAR
        elif tree.label() == 'SBAR' and list_current_depth[-1] == 'SQ':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_SQSBAR = i
                    break    
            if not index_SQSBAR == -1:
                for j in range(index_SQSBAR, index_SQSBAR+len(tree.leaves())):
                    list_SQSBAR[j] = 1        


        # PP - PP
        elif tree.label() == 'PP' and list_current_depth[-1] == 'PP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_PPPP = i
                    break    
            if not index_PPPP == -1:
                for j in range(index_PPPP, index_PPPP+len(tree.leaves())):
                    list_PPPP[j] = 1
                    
        # VP - PP
        elif tree.label() == 'PP' and list_current_depth[-1] == 'VP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_VPPP = i
                    break    
            if not index_VPPP == -1:
                for j in range(index_VPPP, index_VPPP+len(tree.leaves())):
                    list_VPPP[j] = 1
                    
        # NP - PP
        elif tree.label() == 'PP' and list_current_depth[-1] == 'NP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_NPPP = i
                    break    
            if not index_NPPP == -1:
                for j in range(index_NPPP, index_NPPP+len(tree.leaves())):
                    list_NPPP[j] = 1
        
        # NP - NP
        elif tree.label() == 'NP' and list_current_depth[-1] == 'NP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_NPNP = i
                    break    
            if not index_NPNP == -1:
                for j in range(index_NPNP, index_NPNP+len(tree.leaves())):
                    list_NPNP[j] = 1

        # VP - NP
        elif tree.label() == 'NP' and list_current_depth[-1] == 'VP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_VPNP = i
                    break    
            if not index_VPNP == -1:
                for j in range(index_VPNP, index_VPNP+len(tree.leaves())):
                    list_VPNP[j] = 1

        # S - NP
        elif tree.label() == 'NP' and list_current_depth[-1] == 'S':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_SNP = i
                    break    
            if not index_SNP == -1:
                for j in range(index_SNP, index_SNP+len(tree.leaves())):
                    list_SNP[j] = 1
                    
        # VP - VP
        elif tree.label() == 'VP' and list_current_depth[-1] == 'VP':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_VPVP = i
                    break    
            if not index_VPVP == -1:
                for j in range(index_VPVP, index_VPVP+len(tree.leaves())):
                    list_VPVP[j] = 1
        
        # S - VP
        elif tree.label() == 'VP' and list_current_depth[-1] == 'S':
            for i, token in enumerate(temp_sentence):
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                    index_SVP = i
                    break    
            if not index_SVP == -1:
                for j in range(index_SVP, index_SVP+len(tree.leaves())):
                    list_SVP[j] = 1
        
#         elif tree.label() == 'NP':
#             tokens_NP = tree.leaves()

#             for i, token in enumerate(temp_sentence):
#                 if temp_sentence[i] == tokens_NP[0] and temp_sentence[i+len(tokens_NP)-1] == tokens_NP[-1]:
#                     index_NP = i
            
#             if not index_NP == -1:
#                 for j in range(index_NP, index_NP + len(tokens_NP)):
#                     list_NP[j] = 1        
        
        ### For Depth
        # fist tree search
        if depth_cnt == 0:
            current_depth = current_height # depth initialization
            list_current_depth.append(tree.label())
            depth_cnt += 1
        # what is the current depth?
        if current_height == previous_depth-1:
            current_depth = current_height
            list_current_depth.append(tree.label())
        
#         if current_height > previous_height:
#             per_token = []
#         if current_height == 2:
#             phrase_list_until_leaf.append(per_token)
#         else:
#             per_token.append(tree.label())
        
        previous_height = current_height
        previous_depth = current_depth
        

#         print("tree:", tree)
#         print("======> tree.pos()", tree.pos())
#         print("======> tree.height()", tree.height())
#         print("======> tree.label()", tree.label())
#         print("======> tree.leaves()", tree.leaves())
#         print("======> current_depth", current_depth)
#         print('======> current depth list', list_current_depth)
        
#         print('*************************************************************************************************')    
#         print('\n')

    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverseTree(subtree) # recursive

In [33]:
# ############ For Testing

# def traverseTree1(tree):
#     for subtree in tree:
#         if type(subtree) == nltk.tree.Tree:
#             traverseTree(subtree) # recursive

# ttt = 0

# sentence = 'Fischler proposed measures after EU reports.'
# temp = nltk.pos_tag(sentence)
# par_result = parser.tagged_parse(temp)
# temp_sentence = sentence
# initial_tree_parameter(temp_sentence)
    
# for tree in par_result:
#     parse_tree = tree

# # if ttt==1:
# #     print(parse_tree)
    
# # ttt+=1

# traverseTree1(parse_tree)

In [34]:
if using_parsing == 1:
    #########################
    """ For training data """
    #########################
    cnt = 0
    temp_depth_list = []
    temp_n_sibling_list = []
    for i, sentence in enumerate(pre_X_train):
        temp = nltk.pos_tag(sentence)
        par_result = parser.tagged_parse(temp)
        for tree in par_result:
            parse_tree = tree
        #print(parse_tree)

        # For PP, NP, VP,...
        temp_sentence = sentence
        initial_tree_parameter(sentence)
        traverseTree(parse_tree) # For getting tree information

        # For depth, n_sibling...
        depth_list, n_siblings_list = tree_depth_and_n_sibling(parse_tree)
        assert(len(sentence) == len(depth_list))
    #     print(sentence)
    #     print(parse_tree.leaves())
        normalized_li_depth = normalizing_depth_list(depth_list)
        normalized_li_n_siblings = normalizing_n_siblings_list(n_siblings_list)

        for j, token in enumerate(sentence):
            ### Tree depth and sibling
            FE2_X_train[i][j] += FE_depth(j, sentence, normalized_li_depth, window_size_depth) 
            FE2_X_train[i][j] += FE_n_siblings(j, sentence, normalized_li_n_siblings, window_size_n_siblings)     

            ### Phrases-based Features
#             FE2_X_train[i][j] += [list_NPSBAR[j]] # defalut 오직 SBAR만 뽑는다.
#             FE2_X_train[i][j] += [list_VPSBAR[j]]
#             FE2_X_train[i][j] += [list_SSBAR[j]]
#             FE2_X_train[i][j] += [list_SQSBAR[j]]

#             FE2_X_train[i][j] += [list_PPPP[j]] # default 오직 PP만 뽑는다.
#             FE2_X_train[i][j] += [list_VPPP[j]]
#             FE2_X_train[i][j] += [list_NPPP[j]] # sub_refinement_of_object를 추출하는데 도움..

#             FE2_X_train[i][j] += [list_NPNP[j]]
#             FE2_X_train[i][j] += [list_VPNP[j]] 
#             FE2_X_train[i][j] += [list_SNP[j]] 

#             FE2_X_train[i][j] += [list_VPVP[j]] 
#             FE2_X_train[i][j] += [list_SVP[j]]      

        
    #     cnt += 1
    #     print('===================================================================')
    #     if cnt == 1:
    #         break

    ########################
    """ For testing data """
    ########################
    cnt = 0
    for i, sentence in enumerate(pre_X_test):
        temp = nltk.pos_tag(sentence)
        par_result = parser.tagged_parse(temp)
        for tree in par_result:
            parse_tree = tree
        #print(parse_tree)

        # For PP, NP, VP, ...
        temp_sentence = sentence
        initial_tree_parameter(sentence)
        traverseTree(parse_tree) # For getting tree information

        # For dpeth, n_sibling
        depth_list, n_siblings_list = tree_depth_and_n_sibling(parse_tree)
        assert(len(sentence) == len(depth_list))
    #     print(sentence)
    #     print(parse_tree.leaves())
        normalized_li_depth = normalizing_depth_list(depth_list)
        normalized_li_n_siblings = normalizing_n_siblings_list(n_siblings_list)

        for j, token in enumerate(sentence):

            ### Tree-depth, Sibling 
            FE2_X_test[i][j] += FE_depth(j, sentence, normalized_li_depth, window_size_depth) 
            FE2_X_test[i][j] += FE_n_siblings(j, sentence, normalized_li_n_siblings, window_size_n_siblings) 

            ## Phrases-based Festures
#             FE2_X_test[i][j] += [list_NPSBAR[j]]
#             FE2_X_test[i][j] += [list_VPSBAR[j]]
#             FE2_X_test[i][j] += [list_SSBAR[j]]
#             FE2_X_test[i][j] += [list_SQSBAR[j]]

#             FE2_X_test[i][j] += [list_PPPP[j]]
#             FE2_X_test[i][j] += [list_VPPP[j]]
#             FE2_X_test[i][j] += [list_NPPP[j]] 

#             FE2_X_test[i][j] += [list_NPNP[j]]
#             FE2_X_test[i][j] += [list_VPNP[j]] 
#             FE2_X_test[i][j] += [list_SNP[j]] 

#             FE2_X_test[i][j] += [list_VPVP[j]] 
#             FE2_X_test[i][j] += [list_SVP[j]]



    #     print(convert_dummy2(depth_list[j], 0))
    #     print(convert_dummy2(n_siblings_list[j], 1))
    #     print([list_SBAR[j]])
    #     print([list_VPPP[j]])
    #     print([list_NPPP[j]])

    #     cnt += 1
    #     print('===================================================================')
    #     if cnt == 1:
    #         break
       
#     print('===> current feature length', len(FE2_X_train[0][0])-len_pos_feature-len_chunk_feature)
#     len_parsing_feature = len(FE2_X_train[0][0])-len_pos_feature-len_chunk_feature
#     assert(len(FE2_X_train[0][0]) == len(FE2_X_test[0][0]))

## 4. Dependency Parsing

dependency parsing은 spacy 라이브러리도 유명하지만, standford dependency parsing이 좀 더 성능이 좋다는 글을 보고 사용하기로 했다.

In [35]:
# import os
# os.environ['STANFORD_PARSER'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser.jar'
# os.environ['STANFORD_MODELS'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser-3.7.0-models.jar'

In [36]:
# list_dependency = [
#      'nsubj','det','amod','compound','aux','dobj','appos','conj','cc','nmod','case','advcl','mark','nsubjpass',
#      'auxpass','xcomp','advmod','acl','neg','nmod:poss','mwe','nummod','acl:relcl','cop','expl','dep','ccomp',
#      'parataxis','cc:preconj','compound:prt','csubj','nmod:tmod','iobj','det:predet','nmod:npmod','discourse'
# ]

# dict_dependency = { }
# for i, token in enumerate(list_dependency):
#     dict_dependency[token] = i

In [37]:
# ################### Testing
# cntt = 0
# for i, sentence in enumerate(pre_X_train):
#     temp = nltk.pos_tag(sentence)
#     par_result = dep_parser.tagged_parse(temp)
#     for tree in par_result:
#         parse_tree = tree

#     current_sentence = sentence
#     triple_list = list(parse_tree.triples())

#     ### For number of dependencies
#     list_n_dependencies = [1] * len(current_sentence)
#     dic_temp_n_dependencies = { }
#     ### Just counts
#     for triple in triple_list:
#         word = triple[0][0]

#         if not word in dic_temp_n_dependencies:
#             dic_temp_n_dependencies[word] = 1
#         else:
#             dic_temp_n_dependencies[word] += 1
        
#     ### Add dependency counts
#     for m, triple in reversed(list(enumerate(triple_list))): # reversed for loop
#         outbounder = triple[0][0]
#         inbounder = triple[2][0]
#         if outbounder in dic_temp_n_dependencies and inbounder in dic_temp_n_dependencies:
#             # give inbounder count to outbounder count
#             dic_temp_n_dependencies[outbounder] += dic_temp_n_dependencies[inbounder]

#     for n, word in enumerate(current_sentence):
#         if word in dic_temp_n_dependencies:
#             list_n_dependencies[n] = list_n_dependencies[n] + dic_temp_n_dependencies[word]
    
#     print(list_n_dependencies)
#     sentence_n_dependencies = normalized_list_n_dependencies(list_n_dependencies)
    
#         ################################################################################################################ 
#     cntt+=1
#     if cntt==100:
#         break
# #     for j, token in enumerate(current_sentence):
# #         ### Tree depth and sibling

# #         FE2_X_train[i][j] += [sentence_n_dependencies[j]] # converted into list

In [38]:
# if using_dependency == 1:
#     #####################
#     """ Training Data """
#     #####################
#     cntt = 0
#     for i, sentence in enumerate(pre_X_train):
#         temp = nltk.pos_tag(sentence)
#         par_result = dep_parser.tagged_parse(temp)
#         for tree in par_result:
#             parse_tree = tree

#         current_sentence = sentence
#         triple_list = list(parse_tree.triples())

#         ### For number of dependencies
#         list_n_dependencies = [1] * len(current_sentence)
#         dic_temp_n_dependencies = { }
#         ### Just counts
#         for triple in triple_list:
#             word = triple[0][0]

#             if not word in dic_temp_n_dependencies:
#                 dic_temp_n_dependencies[word] = 1
#             else:
#                 dic_temp_n_dependencies[word] += 1
#         ### Add dependency counts
#         for m, triple in reversed(list(enumerate(triple_list))): # reversed for loop
#             outbounder = triple[0][0]
#             inbounder = triple[2][0]
#             if outbounder in dic_temp_n_dependencies and inbounder in dic_temp_n_dependencies:
#                 # give inbounder count to outbounder count
#                 dic_temp_n_dependencies[outbounder] += dic_temp_n_dependencies[inbounder]

#         for n, word in enumerate(current_sentence):
#             if word in dic_temp_n_dependencies:
#                 list_n_dependencies[n] = list_n_dependencies[n] + dic_temp_n_dependencies[word]

#         normalized_li_n_dep = normalized_list_n_dependencies(list_n_dependencies)

#         ################################################################################################################ 
#         for j, token in enumerate(current_sentence):
#             ### Tree depth and sibling

#             FE2_X_train[i][j] += FE_n_dependencies(j, sentence, normalized_li_n_dep, window_size_n_dep)

# #             FE2_X_train[i][j] += FE_dependency(j, sentence, triple_list, 0, dict_dependency)
# #             FE2_X_train[i][j] += FE_dependency(j, sentence, triple_list, 1, dict_dependency)
        
#     #     cntt += 1
#     #     if cntt == 1:
#     #         break

#     ####################
#     """ Testing Data """
#     ####################
#     cntt = 0
#     for i, sentence in enumerate(pre_X_test):
#         temp = nltk.pos_tag(sentence)
#         par_result = dep_parser.tagged_parse(temp)
#         for tree in par_result:
#             parse_tree = tree

#         current_sentence = sentence
#         triple_list = list(parse_tree.triples())

#         ### For number of dependencies
#         list_n_dependencies = [1] * len(current_sentence)
#         dic_temp_n_dependencies = { }
#         ### Just counts
#         for triple in triple_list:
#             word = triple[0][0]

#             if not word in dic_temp_n_dependencies:
#                 dic_temp_n_dependencies[word] = 1
#             else:
#                 dic_temp_n_dependencies[word] += 1
#         ### Add dependency counts
#         for m, triple in reversed(list(enumerate(triple_list))): # reversed for loop
#             outbounder = triple[0][0]
#             inbounder = triple[2][0]
#             if outbounder in dic_temp_n_dependencies and inbounder in dic_temp_n_dependencies:
#                 # give inbounder count to outbounder count
#                 dic_temp_n_dependencies[outbounder] += dic_temp_n_dependencies[inbounder]

#         for n, word in enumerate(current_sentence):
#             if word in dic_temp_n_dependencies:
#                 list_n_dependencies[n] = list_n_dependencies[n] + dic_temp_n_dependencies[word]

#         normalized_li_n_dep = normalized_list_n_dependencies(list_n_dependencies)

#         ################################################################################################################ 
#         for j, token in enumerate(current_sentence):
#             ### Tree depth and sibling

#             FE2_X_test[i][j] += FE_n_dependencies(j, sentence, normalized_li_n_dep, window_size_n_dep)

# #             FE2_X_test[i][j] += FE_dependency(j, sentence, triple_list, 0, dict_dependency)
# #             FE2_X_test[i][j] += FE_dependency(j, sentence, triple_list, 1, dict_dependency)
        
#     #     cntt += 1
#     #     if cntt == 1:
#     #         break


In [39]:
### Triple List
# [(('include', 'VB'), 'nsubj', ('section', 'NN')),
#  (('section', 'NN'), 'det', ('A', 'DT')),
#  (('section', 'NN'), 'amod', ('clinical', 'JJ')),
#  (('section', 'NN'), 'compound', ('lab', 'NN')),
#  (('include', 'VB'), 'aux', ('shall', 'MD')),
#  (('include', 'VB'), 'dobj', ('name', 'NN')),
#  (('name', 'NN'), 'det', ('the', 'DT')),
#  (('name', 'NN'), 'amod', ('clinical', 'JJ')),
#  (('name', 'NN'), 'compound', ('site', 'NN')),
#  (('name', 'NN'), 'appos', ('class', 'NN')),
#  (('class', 'NN'), 'det', ('the', 'DT')),
#  (('class', 'NN'), 'conj', ('instructor', 'NN')),
#  (('class', 'NN'), 'conj', ('day', 'NN')),
#  (('class', 'NN'), 'cc', ('and', 'CC')),
#  (('class', 'NN'), 'conj', ('time', 'NN')),
#  (('class', 'NN'), 'nmod', ('lab', 'NN')),
#  (('lab', 'NN'), 'case', ('of', 'IN')),
#  (('lab', 'NN'), 'det', ('the', 'DT'))]

In [40]:
# cntt = 0
# for i, sentence in enumerate(pre_X_test):
#     temp = nltk.pos_tag(sentence)
#     par_result = parser.tagged_parse(temp)
#     for tree in par_result:
#         parse_tree = tree
#     print(parse_tree)
    
#     cntt += 1
#     if cntt == 100:
#         break

## Write out files

In [41]:
dump(FE2_X_train, 'FE2_X_train')
dump(FE2_X_test, 'FE2_X_test')
print('===> Total feature length =', len(FE2_X_train[0][0]), len(FE2_X_test[0][0]))

===> Total feature length = 22 22


In [42]:
# # 1. Ngram tagger
# from nltk.tag import DefaultTagger
# from nltk.tag import UnigramTagger
# from nltk.tag import BigramTagger
# from nltk.tag import TrigramTagger
# from nltk.tag import NgramTagger

# from nltk.corpus import treebank
# train_sents = treebank.tagged_sents()

# def backoff_tagger(train_sents, tagger_classes, backoff=None):  
#     for cls in tagger_classes:    
#         backoff = cls(train_sents, backoff=backoff)    
#     return backoff 

# backoff = DefaultTagger('NN')
# tagger = backoff_tagger(train_sents, [UnigramTagger, BigramTagger, TrigramTagger], backoff=backoff) 

# # 2. tnt Tagger
# from nltk.tag import tnt 
# from nltk.tag import DefaultTagger
# unk = DefaultTagger('NN') 
# tnt_tagger = tnt.TnT() 
# tnt_tagger = tnt.TnT(unk=unk, Trained=True)
# tnt_tagger.train(train_sents)

# # predefined tnt tagger
# from nltk.tag.sequential import ClassifierBasedPOSTagger
# tagger = ClassifierBasedPOSTagger(train=train_sents) 

# # tagger can be saved
# import pickle 
# f = open('tagger.pickle', 'wb') 
# pickle.dump(tagger, f) 
# f.close() 
# f = open('tagger.pickle', 'rb') 
# tagger = pickle.load(f)

In [43]:
# yD0 = []
# yD1 = []
# yD2 = []
# yD3 = []
# yD4 = []
# yD5 = []
# yD6 = [] 
# yD7 = []
# yD8 = []
# yD9 = []
# yD10 = []
# yD11 = []
# yD12 = []
# yD13 = []
# yD14 = []
# yD15 = []
# yD16 = []
# yS0 = []
# yS1 = []
# yS2 = []
# yS3 = []
# yS4 = []
# yS5 = []
# yS6 = [] 
# yS7 = []
# yS8 = []
# yS9 = []
# yS10 = []
# yS11 = []
# yS12 = []
# yS13 = []
# yS14 = []
# yS15 = []
# yS16 = []
        # For counting depth, n_sibling
#         if Y_train[i][j] == 0:
#             yD0.append(depth_list[j])
#             yS0.append(n_siblings_list[j])
#         elif Y_train[i][j] == 1:
#             yD1.append(depth_list[j])
#             yS1.append(n_siblings_list[j])        
#         elif Y_train[i][j] == 2:
#             yD2.append(depth_list[j])
#             yS2.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 3:
#             yD3.append(depth_list[j])
#             yS3.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 4:
#             yD4.append(depth_list[j])
#             yS4.append(n_siblings_list[j])        
#         elif Y_train[i][j] == 5:
#             yD5.append(depth_list[j])
#             yS5.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 6:
#             yD6.append(depth_list[j])
#             yS6.append(n_siblings_list[j])
#         elif Y_train[i][j] == 7:
#             yD7.append(depth_list[j])
#             yS7.append(n_siblings_list[j])        
#         elif Y_train[i][j] == 8:
#             yD8.append(depth_list[j])
#             yS8.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 9:
#             yD9.append(depth_list[j])
#             yS9.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 10:
#             yD10.append(depth_list[j])
#             yS10.append(n_siblings_list[j])        
#         elif Y_train[i][j] == 11:
#             yD11.append(depth_list[j])
#             yS11.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 12:
#             yD12.append(depth_list[j])
#             yS12.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 13:
#             yD13.append(depth_list[j])
#             yS13.append(n_siblings_list[j])        
#         elif Y_train[i][j] == 14:
#             yD14.append(depth_list[j])
#             yS14.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 15:
#             yD15.append(depth_list[j])
#             yS15.append(n_siblings_list[j]) 
#         elif Y_train[i][j] == 16:
#             yD16.append(depth_list[j])
#             yS16.append(n_siblings_list[j])

In [44]:
# # Frequency for depth and sibling
# unfolded_temp_depth_list, unfolded_temp_n_sibling_list = unfold_data(temp_depth_list, temp_n_sibling_list)
# a = np.array(unfolded_temp_depth_list)
# b = np.bincount(a)
# ii = np.nonzero(b)[0]
# np.vstack((ii,b[ii])).T